# Taller de Física Computacional

Carlos Ruestes / Cristián Sánchez - Taller de Física Computacional - FCEN - UNCUYO

# Sesión 14: Modos normales de una cadena unidimensional de dímeros

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import scipy.optimize as opt

Al igual que en los casos anteriores las partículas interacitúan por un potencial de Lennard-Jones, en este caso se ha disminuído significativamente el parámetro $\epsilon$ para dar cuenta de la interacción débil entre dímeros enlazados químicamente:

In [ ]:
EPSILON = 0.001
SIGMA = 3.5

In [ ]:
def vlj(r):
    return EPSILON*((SIGMA / r)**12 - 2*(SIGMA / r)**6) 

In [ ]:
plt.plot(np.linspace(3,5,100),vlj(np.linspace(3,5,100)))

In [ ]:
def d_vlj(r):
    return EPSILON*( 12*SIGMA**6*r**(-7) -12*SIGMA**12*r**(-13) )

In [ ]:
def dd_vlj(r):
    return EPSILON*(-84*SIGMA**6*r**(-8)+156*SIGMA**12*r**(-14))

Al potencial de Lennard-Jones ahora sumamos una interacción de tipo Morse con una distancia de equilibrio menor a la de Lennard-Jones para representar el enlace químico entre dímeros:

In [ ]:
De = 30
a = 2
re = 1.2

In [ ]:
def morse(r):
    return De * (1 - np.exp(-a*(r-re)))**2 - De

In [ ]:
plt.plot(np.linspace(0.8,5,100),morse(np.linspace(0.8,5,100)))

In [ ]:
def d_morse(r):
    return (2*a*De*(1 - np.exp(-(a*(r - re)))))/np.exp(a*(r - re))

In [ ]:
def dd_morse(r):
    return De*((2*a**2)/np.exp(2*a*(r - re)) - 
     (2*a**2*(1 - np.exp(-(a*(r - re)))))/np.exp(a*(r - re)))

Como hemos hecho antes definimos el número total de partículas, la función de distancia en la recta, energía total, su derivada respecto a la coordenada de una partícula y el gradiente:

In [ ]:
NPART = 50

In [ ]:
def rij(xi,xj):
    return abs(xj - xi)

In [ ]:
def d_rij(xi,xj,k):
    if (xi - xj) > 0.0 and k == 1:
        return 1.0
    elif (xi - xj) < 0.0 and k == 1:
        return -1.0
    elif (xi - xj) > 0.0 and k == 2:
        return -1.0
    elif (xi - xj) < 0.0 and k == 2:
        return  1.0
    else:
        raise ValueError('k sólo puede ser 1 o 2.')

In [ ]:
def dd_rij(xi,xj,k,l):
    return 0.0

In [ ]:
K = 1000

In [ ]:
def energy(xs,L):
    energy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                energy += vlj(rij(xs[i],xs[j]))
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j) and (i%2 == 0) and (j==i+1):
                energy += morse(rij(xs[i],xs[j]))
    energy += 0.5 * K * (xs[0] - 0)**2
    energy += 0.5 * K * (xs[NPART-1] - L)**2
    return 0.5 * energy

In [ ]:
def d_energy(xs,k,L):
    denergy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                if k == i:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],1)
                elif k == j:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],2) 
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j) and (i%2 == 0) and (j==i+1):
                if k == i:
                    denergy += d_morse(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],1)
                elif k == j:
                    denergy += d_morse(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],2) 
    if (k == 0):
        denergy += K * (xs[0] - 0)
    if (k == NPART-1):
        denergy += K * (xs[NPART-1] - L)
    return denergy

In [ ]:
def grad_energy(xs,L):
    gradient = np.zeros((NPART))
    for i in range(0,NPART):
        gradient[i] = d_energy(xs,i,L)
    return gradient

Construimos una geometría inicial para comenzar la optimización teniendo en cuenta que ahora hay dos distancias de equilibrio:

In [ ]:
xcoords = np.zeros(NPART)
xcoords[0] = 0.0
for i in range(1,NPART):
    if (i%2==1):
        xcoords[i] = xcoords[i-1] + 1.87
    else:
        xcoords[i] = xcoords[i-1] + 3.45

In [ ]:
y = np.zeros_like(xcoords) + 0.0
plt.figure(figsize=(10.0,1.0))
plt.plot(xcoords,y,marker = "o")

In [ ]:
L = 129.48655989775992 * 1.008

In [ ]:
res = opt.minimize(energy, xcoords, method='BFGS', tol = 1e-6, jac=grad_energy, args=(L,),
                   options={'disp': True})
res.success

In [ ]:
equilibrio = res.x

In [ ]:
xcoords = equilibrio

In [ ]:
plt.plot(grad_energy(equilibrio,L))

In [ ]:
y = np.zeros_like(equilibrio) + 0.0
plt.figure(figsize=(10.0,1.0))
plt.plot(equilibrio,y,marker = "o")

In [ ]:
- K * equilibrio[0]

In [ ]:
- K * (equilibrio[NPART-1] - L)

In [ ]:
equilibrio[1]-equilibrio[0]

In [ ]:
equilibrio[2]-equilibrio[1]

In [ ]:
equilibrio[3]-equilibrio[2]

In [ ]:
largo = equilibrio[NPART-1] - equilibrio[0]
print("El largo de la cadena es ",largo)

In [ ]:
def dd_energy(xs,k,l):
    ddenergy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                if (k == i) and (l == i):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,1)
                elif (k == i) and (l == j):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,1)
                elif (k == j) and (l == i):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,2)
                elif (k == j) and (l == j):
                    ddenergy += dd_vlj(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_vlj(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,2)
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j) and (i%2 == 0) and (j==i+1):
                if (k == i) and (l == i):
                    ddenergy += dd_morse(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_morse(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,1)
                elif (k == i) and (l == j):
                    ddenergy += dd_morse(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                +d_morse(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,1)
                elif (k == j) and (l == i):
                    ddenergy += dd_morse(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],1)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_morse(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],1,2)
                elif (k == j) and (l == j):
                    ddenergy += dd_morse(rij(xcoords[i],xcoords[j])) \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                *d_rij(xcoords[i],xcoords[j],2)    \
                                +d_morse(rij(xcoords[i],xcoords[j])) \
                                *dd_rij(xcoords[i],xcoords[j],2,2) 
    if (k==0) and (l==0):
        ddenergy += K
    elif (k==NPART-1) and (l==NPART-1):
        ddenergy += K
    return ddenergy

In [ ]:
k_matrix = np.zeros((NPART,NPART))
for i in range(0,NPART):
    for j in range(0,NPART):
        k_matrix[i,j] = dd_energy(xcoords,i,j) 

In [ ]:
plt.imshow(k_matrix)

In [ ]:
evals, evecs = np.linalg.eigh(k_matrix)

In [ ]:
evals

In [ ]:
plt.plot(np.sqrt(evals[0:23]),marker="o")

In [ ]:
plt.plot(np.sqrt(evals[25:NPART-2]),marker="o")

In [ ]:
plt.plot(np.sqrt(evals),marker="o")
plt.xlim(0,6)
plt.ylim(0,0.06)

In [ ]:
plt.plot(evecs[:,0])
plt.plot(evecs[:,1])
plt.plot(evecs[:,2])
plt.plot(evecs[:,3])

In [ ]:
plt.plot(evecs[:,25])

In [ ]:
print(np.sqrt(evals[0]))
print(np.sqrt(evals[1]))
print(np.sqrt(evals[2]))
print(np.sqrt(evals[3]))

In [ ]:
print(np.sqrt(evals[25]))
print(np.sqrt(evals[26]))
print(np.sqrt(evals[27]))
print(np.sqrt(evals[28]))

In [ ]:
plt.plot(evecs[:,23])
plt.plot(evecs[:,24])
plt.plot(evecs[:,NPART-1])
plt.plot(evecs[:,NPART-2])

In [ ]:
print(np.sqrt(evals[23]))
print(np.sqrt(evals[24]))
print(np.sqrt(evals[25]))
print(np.sqrt(evals[26]))